Calculate `x mod 2` for an array of numbers and compare multithreaded scheduler, workers in a pseudo-distributed cluster, and multiple cloud-based workers.

Example similar to http://docs.continuum.io/anaconda-cluster/examples/spark-basic

In [1]:
import dask.array as da

In [2]:
from distributed import Executor
ex_local = Executor('127.0.0.1:8786')
ex_cloud = Executor('54.164.90.63:8786')

In [3]:
A = da.arange(1e7, chunks=1e3)

In [4]:
%time list(zip(A.compute(), (A % 2).compute()))[:10]

CPU times: user 6.48 s, sys: 962 ms, total: 7.44 s
Wall time: 7.17 s


[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1)]

In [5]:
%time list(zip(A.compute(get=ex_local.get), (A % 2).compute(get=ex_local.get)))[:10]

CPU times: user 21.4 s, sys: 8.51 s, total: 29.9 s
Wall time: 2min 12s


[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1)]

The above line was running in a pseudo-distributed cluster with three workers on the same machine as the multithreaded scheduler.

In [6]:
%time list(zip(A.compute(get=ex_cloud.get), (A % 2).compute(get=ex_cloud.get)))[:10]

CPU times: user 26 s, sys: 12.5 s, total: 38.6 s
Wall time: 1min 38s


[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 1),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1)]

The above calculation was performed on three compute nodes running on Amazon EC2 (m4.xlarge). Additional time was required to move the data across the remote network. The same calculation run on the head node of the cluster took 1 min 7 s.